## Python environment

With python3.9 the following commands in the command line should create an environment sufficient to run this notebook.

```
    envname="Qmin-vis-env"   
    python3 -m venv "$envname"
    source "$envname"/bin/activate  
    python3 -m pip install pandas pyvistaqt jupyterlab PyQt5 tqdm
    python3 -m ipykernel install --name="$envname" --user
```    

(This assumes `python3` points to an installation of python3.9. I'm not sure about other versions yet but it looks like python3.10 is not working as of 10/27/21. You may have to open jupyter lab as `python3 -m jupyter lab`.)

In [1]:

# from PySide2 import QtGui
from qtpy import QtWidgets as qw
import qtpy.QtCore as Qt
from pandas import read_csv
import pyvista as pv
import pyvistaqt as pvqt
import numpy as np
# from PySide6 import QtWidgets as qw
# from PyQt5 import QtWidgets as qw

In [2]:
"""Get that file!"""

# !scp dbeller@skyrmion.ucmerced.edu:open-Qmin-clones/clone7/tetstGUIbdys_x0y0z0.txt ./
dat = np.array(read_csv("tetstGUIbdys_x0y0z0.txt", sep="\t", header=None))

In [381]:
class Qmin_plot(pvqt.BackgroundPlotter):
    def __init__(self, oQm_data, user_settings={}):
        super().__init__()#window_size=settings["window_size"])
        self.set_settings(user_settings)
        self.widgets = {}

        self.Lx, self.Ly, self.Lz = [ int(item+1) for item in oQm_data[-1,:3] ]
        self.meshdata_from_file(oQm_data)        
        
        ## global options
        pv.global_theme.title = "open-Qmin visualization with pyvistaqt"
        

        self.enable_eye_dome_lighting() 
        self.rescale_lights_intensity(16)
        self.renderer.add_axes(interactive=True, color='black') # xyz axes arrows
        self.renderer.set_background("white")
        
        
      
        self.add_mesh(
            self.fullmesh.contour([0.5], scalars="nematic_sites"), 
            show_scalar_bar=False, 
            color=self.settings["boundaries_color"],
            name="boundaries",
            ambient=1,
    #         specular=1, ambient=1, # lighting
    #         smooth_shading=True, # very smooth! 
            pbr=True, metallic=0.5, roughness=0.25, diffuse=1
            )   
 
        
#         self.visibilities = {
#             "defects": True,
#             "L1_isosurface": False,
#             "L2_isosurface": False,
#             "L3_isosurface": False,
#             "L6_isosurface": False,
#             "K1_isosurface": False,
#             "K2_isosurface": False,
#             "K3_isosurface": False,
#             "director": True,
#             "slice_plane": True
#         }
        
        self.QSliders_toolbar = qw.QToolBar('QSliders')
        self.app_window.addToolBar(Qt.Qt.LeftToolBarArea, self.QSliders_toolbar)


        self.QSliders={}
        
        nres = self.settings["director_resolution"]        
        self.n_res_slider_label = qw.QLabel()
        self.n_res_slider = qw.QSlider(Qt.Qt.Horizontal)
        self.n_res_slider.setMinimum(1)
        self.n_res_slider.setMaximum(int(max(1,max(np.array([self.Lx,self.Ly,self.Lz])/6))))
        self.n_res_slider.setValue(nres)        


        self.QSliders_toolbar.addWidget(self.n_res_slider_label)    
        self.QSliders_toolbar.addWidget(self.n_res_slider)
        self.n_res_slider.valueChanged.connect(self.make_coarsemesh)            
    
        actor_name="defects"
        self.defects_label, self.set_defect_S, self.QSliders[actor_name] = self.add_QSlider(
            self.update_defects,
            self.QSliders_toolbar,
            actor_name,            
            scalars=self.fullmesh["order"],
            min_val=0,
            label_txt="S"
        )
        
        self.update_L1 = lambda value: self.update_Li(value, 1)
        self.update_L2 = lambda value: self.update_Li(value, 2)
        self.update_L3 = lambda value: self.update_Li(value, 3)        
        self.update_L6 = lambda value: self.update_Li(value, 6)        

        self.update_K1 = lambda value: self.generic_slider_callback(
            f"K1_isosurface", f"energy_K1", value, self.settings[f"energy_K1_color"]
        )
        i=2
        self.update_K2 = lambda value: self.generic_slider_callback(
            f"K2_isosurface", f"energy_K2", value, self.settings[f"energy_K2_color"]
        )
        i=3
        self.update_K3 = lambda value: self.generic_slider_callback(
            f"K3_isosurface", f"energy_K3", value, self.settings[f"energy_K3_color"]
        )
        
        i=1
        actor_name="L1_isosurface"
        self.L1_label, self.set_L1, self.QSliders[actor_name] = self.add_QSlider(
                self.update_L1,
                self.QSliders_toolbar,
                actor_name,
                scalars=self.fullmesh[f"energy_L{i}"],
                min_val=0,
                label_txt=f"L_{i}",
            )
        
        i=2
        actor_name="L2_isosurface"
        self.L2_label, self.set_L2, self.QSliders[actor_name] = self.add_QSlider(
                self.update_L2,
                self.QSliders_toolbar,
                actor_name,
                scalars=self.fullmesh[f"energy_L{i}"],
                min_val=0,
                label_txt=f"L_{i}",
            )
        i=3
        actor_name="L3_isosurface"        
        self.L3_label, self.set_L3, self.QSliders[actor_name] = self.add_QSlider(
                self.update_L3,
                self.QSliders_toolbar,
                actor_name,
                scalars=self.fullmesh[f"energy_L{i}"],
                label_txt=f"L_{i}",
            )   
        i=6
        actor_name="L6_isosurface"        
        self.L6_label, self.set_L6, self.QSliders[actor_name] = self.add_QSlider(
                self.update_L6,
                self.QSliders_toolbar,
                actor_name,
                scalars=self.fullmesh[f"energy_L{i}"],
                label_txt=f"L_{i}",
            )      

        actor_name="K1_isosurface"        
        self.K1_label, self.set_K1, self.QSliders[actor_name] = self.add_QSlider(
                self.update_K1,
                self.QSliders_toolbar,
                actor_name,
                scalars=self.fullmesh[f"energy_K1"],
                label_txt=f"K_1",
            )          
        
        actor_name="K2_isosurface"        
        self.K2_label, self.set_K2, self.QSliders[actor_name] = self.add_QSlider(
                self.update_K2,
                self.QSliders_toolbar,
                actor_name,
                scalars=self.fullmesh[f"energy_K2"],
                label_txt=f"K_2",
            )          
        
        actor_name="K3_isosurface"        
        self.K3_label, self.set_K3, self.QSliders[actor_name] = self.add_QSlider(
                self.update_K3,
                self.QSliders_toolbar,
                actor_name,
                scalars=self.fullmesh[f"energy_K3"],
                label_txt=f"K_3",
            )          
                
        self.make_coarsemesh(self.settings["director_resolution"])
         
        # initialize the actors
        self.update_defects(0.3)
        self.update_L1(np.average(self.fullmesh["energy_L1"]))
        self.update_L2(np.average(self.fullmesh["energy_L2"]))
        self.update_L3(np.average(self.fullmesh["energy_L3"]))
        self.update_L6(np.average(self.fullmesh["energy_L6"]))
        self.update_K1(np.average(self.fullmesh["energy_K1"]))
        self.update_K2(np.average(self.fullmesh["energy_K2"]))        
        self.update_K3(np.average(self.fullmesh["energy_K3"]))

        for i in [1,2,3,6]:
            self.renderer.actors[f"L{i}_isosurface"].SetVisibility(False)
        for i in [1,2,3]:
            self.renderer.actors[f"K{i}_isosurface"].SetVisibility(False)

        self.plane_widget_toggle("director_slice")()
        
 
        
        self.add_mesh(self.fullmesh.outline(), color='black') # bounding box)
        
        
        
        self.toggle_menu = self.main_menu.addMenu('Toggle')
        for actor_name in self.renderer.actors:
            menu_action = self.toggle_menu.addAction(actor_name, 
                                  self.generic_menu_toggle(actor_name))
            menu_action.setCheckable(True)
            is_visible = self.renderer.actors[actor_name].GetVisibility()
            menu_action.setChecked(is_visible)
#             self.visibilities[actor_name] = is_visible
        menu_action = self.toggle_menu.addAction("director_slice", self.plane_widget_toggle("director_slice"))
        menu_action.setCheckable(True)
        menu_action.setChecked(True)
            


    def Q33_from_Q5(self, Q5):
        (Qxx, Qxy, Qxz, Qyy, Qyz) = Q5.T
        Qmat = np.moveaxis(np.array([ 
                [Qxx, Qxy, Qxz],
                [Qxy, Qyy, Qyz],
                [Qxz, Qyz, -Qxx-Qyy]
                ]), -1, 0)
        return Qmat

    def n_from_Q(self, Qmat):
        """Get director from 3x3-matrix Q-tensor data"""
        evals, evecs = np.linalg.eigh(Qmat)    
        return evecs[:,:,2]            

    def set_settings(self, user_settings):
        self.settings = {
            "boundaries_color":"gray",
            "director_color":"red",
            "director_resolution":2, 
            "default_defect_S":0.3, # initialization order value for defect isosurfaces
            "defect_color":(37/256,150/256,190/256),
            "checkbox_size":50, # size of toggle boxes in pixels
            "checkbox_spacing":10, # spacing between toggle boxes in pixels
            "window_size":(1200,800), # window size in pixels
            "cylinder_resolution":8, # angular resolution for each cylindrical rod; larger values look nicer but take longer to compute
            "slice_plane_color":"lightyellow", # set to None to use slice_color_function instead    
            "slice_cmap":"cividis", # color map for use with slice_color_function
            "slice_color_function":(lambda slc: np.abs(slc["director"][:,0])), # optionally color slice plane by some function of director or order
            "plane_widget_color":"orange",
            "energy_L1_color":"yellow",
            "energy_L2_color":"pink",
            "energy_L3_color":"purple",
            "energy_L6_color":"gray",
            "energy_K1_color":"gray",
            "energy_K2_color":"gray",
            "energy_K3_color":"gray"
        }
        for key, value in zip(user_settings.keys(), user_settings.values()):
            self.settings[key] = value

    def meshdata_from_file(self, dat):

        # name the data columns:
        self.coords = dat[:,:3]
        self.Qdata = dat[:,3:8]
        self.Q33 = self.Q33_from_Q5(self.Qdata) # 3x3 Q matrix
        self.site_types = dat[:,8]
        self.order = dat[:,9]

        # grid for pyvista:
        self.fullmesh = pv.UniformGrid((self.Lx,self.Ly,self.Lz)) 
        # Order (defects) data:
        self.order[self.site_types>0] = np.max(self.order) # Don't plot defects inside objects
        self.fullmesh["order"] = self.order

        # director data:
        self.Qdata[self.site_types>0] = 0. # Don't bother calculating eigenvectors inside objects
        self.fullmesh["director"] = self.n_from_Q(self.Q33)

        # boundaries:
        self.fullmesh["nematic_sites"] = 1*(self.site_types<=0)        

        
        self.Q33_xyz = self.Q33.reshape((self.Lx,self.Ly,self.Lz,3,3))        
        self.diQjk = np.moveaxis(
            np.array(
                [np.roll(self.Q33_xyz,-1, axis=i) 
                 - np.roll(self.Q33_xyz,1, axis=i) 
                 for i in range(3)]
            ),
            0, -3
        )
        
        # energy:
        self.fullmesh["energy_L1"] = np.sum(
            self.diQjk**2, axis=(-1,-2,-3)).flatten()
        self.fullmesh["energy_L2"] = np.sum(
            (np.einsum("...iij->...j", self.diQjk))**2, 
            axis=-1
        ).flatten() # djQij dkQik = djQji dkQki = sum_i[(djQji)^2]
        self.fullmesh["energy_L6"] = np.einsum(
            "...ij,...ikl,...jkl", self.Q33_xyz, self.diQjk, self.diQjk).flatten()
        self.fullmesh["energy_L3"] = np.einsum(
            "...ijk,...kij", self.diQjk, self.diQjk).flatten() 
        self.fullmesh["energy_L24"] = self.fullmesh["energy_L3"] - self.fullmesh["energy_L2"] # diQjk dkQij - diQij dkQjk 
        for i in [1,2,3,6,24]:
            self.fullmesh[f"energy_L{i}"] *= 1*(self.site_types==0)        
        L1 = self.fullmesh["energy_L1"]
        L2 = self.fullmesh["energy_L2"]
        L3 = self.fullmesh["energy_L3"]
        L6 = self.fullmesh["energy_L6"]
        S = self.fullmesh["order"]
        self.fullmesh["energy_K1"] = 2/(9*S*S) * (-L1/3 + 2*L2 -2/(3*S)*L6)
        self.fullmesh["energy_K2"] = 2/(9*S*S) * (L1 - 2*L3)
        self.fullmesh["energy_K3"] = 2/(9*S*S) * (L1/3 + 2/(3*S)*L6)        
                      
            
    def make_coarsemesh(self, nres):
        self.settings["director_resolution"] = nres
        self.coarsemesh = self.fullmesh.probe(
            pv.UniformGrid(tuple([ int(item/nres) for item in [self.Lx, self.Ly, self.Lz] ]), 
                           (nres,)*3
                          )
        )   
        try:
            self.n_res_slider_label.setText(f"n_res: {nres}")                                        
        except AttributeError:
            pass
        
        self.director_slice_func = self.make_director_slice_func()
        for i in range(2):
            self.plane_widget_toggle("director_slice")()
            
    def make_director_slice_func(self):        

        def director_slice_func(normal, origin):
            """make glyph plot and transparent plane for director field slice"""
            slc = self.coarsemesh.slice(normal=normal, origin=origin)
            cylinders = slc.glyph(orient="director", scale="nematic_sites",
                factor=self.settings["director_resolution"],
                geom=pv.Cylinder(
                    radius=0.2, height=1, 
                    resolution=self.settings["cylinder_resolution"]
                ),  
                tolerance=None
    #             progress_bar=True
            )
            try:
                director_vis = self.renderer.actors["director"].GetVisibility()                
                slice_plane_vis = self.renderer.actors["slice_plane"].GetVisibility()
            except KeyError:
                director_vis = 1
                slice_plane_vis = 1
            self.renderer.actors["director"] = self.add_mesh(
                cylinders, 
                color=self.settings["director_color"], 
    #             specular=1, ambient=0.35, 
                pbr=True, metallic=0.5, roughness=0.25, diffuse=1,
                name="director" # "name" == actor's name so old actor is replaced
            )
            self.renderer.actors["director"].SetVisibility(director_vis)

            self.renderer.actors["slice_plane"] = self.add_mesh(
                slc, opacity=0.01, 
                ambient=1, diffuse=0, specular=0, # glows, doesn't reflect
                color=self.settings["slice_plane_color"], 
                scalars=(self.settings["slice_color_function"](slc) 
                    if self.settings["slice_plane_color"] is None 
                    else None # use slice_color_function only if slice_plane_color==None
                ),
                cmap=self.settings["slice_cmap"],
                name="slice_plane" # "name" == actor's name so old actor is replaced
            )
            self.renderer.actors["slice_plane"].SetVisibility(slice_plane_vis)
#             except ValueError:
#                 pass
            
        return director_slice_func
      
        
    def generic_slider_callback(self, actor_name, scalars, contour_value, 
        color=None, mesh_color_scalars=None, clim=None, cmap=None):
        """generic callback function for slider controlling isosurfaces"""
        kwargs={
            "show_scalar_bar":False,
            "specular":1, "specular_power":20,
            "ambient":0.5, "diffuse":1,
            "smooth_shading":True,
            "name":actor_name,
            "pbr":True, 
            "metallic":0,
            "roughness":0.25,
            "diffuse":1,            
        }
        if mesh_color_scalars is not None:
            kwargs["scalars"] = mesh_color_scalars
            if clim is not None:
                kwargs["clim"] = clim
            if cmap is not None:
                kwargs["cmap"] = cmap            
        elif color is not None:
            kwargs["color"] = color
        
        try:
            self.renderer.actors[actor_name] = self.add_mesh(
                self.fullmesh.contour(
                    [contour_value], scalars=scalars
                ), **kwargs            
            ) 
        except ValueError:
            pass
        
        
     
    def update_defects(self, dthresh):
        return self.generic_slider_callback(
            "defects", "order", dthresh, color=None, mesh_color_scalars="energy_L24", clim=[-1,1], cmap="jet"
        )
    
    def update_Li(self, Li_value, i, vis=True):
        return self.generic_slider_callback(
            f"L{i}_isosurface", f"energy_L{i}", Li_value, self.settings[f"energy_L{i}_color"]
        )
    
    def plane_widget_toggle(self, widget_name):
        def return_function():
            if widget_name in self.widgets.keys():
                vis = 1-(self.widgets[widget_name][0] != 0)
            else:
                vis = True
            if vis:
                self.widgets[widget_name] = (True, 
                    self.add_plane_widget(self.director_slice_func, 
                        factor=1.1, 
                        color=self.settings["plane_widget_color"],
                        tubing=False
                    )
                )
            else:
                self.widgets[widget_name] = (False, self.clear_plane_widgets())
                
        return return_function
    
    
    ## the sliders
   
    def generic_isosurface_slider(self, callback_function, scalars, pointa, pointb, init_value=None, title=None, color=None):
        min_val = 1.01*np.min(self.fullmesh[scalars])
        max_val = 0.99*np.max(self.fullmesh[scalars])
        if min_val > max_val:
            tmp = min_val
            min_val = max_val
            max_val = tmp
        if init_value is None:
            init_value = 0.5*(min_val+max_val)      
        if color is not None:
            pv.global_theme.slider_styles.modern.slider_color = color
        self.add_slider_widget(
            callback_function,
            [min_val, max_val], # value range
            value=init_value,
            color="black", # text color
            fmt="%0.3f", # text format
            pointa=pointa,
            pointb=pointb,
            style="modern",
            title=title
        )
        
    def add_QSlider(self, update_method, toolbar, actor_name, num_divs=100, 
                    init_val=30, min_val = None, max_val = None, scalars=None, label_txt=None):
        def slider_formula(slider_value):
            return min_val + (max_val-min_val)*slider_value/100         
        def external_update(float_value):
            slider_value = int( 100*(float_value-min_val)/(max_val-min_val))
            slider.SetValue(slider_value)
        
        slider = qw.QSlider(Qt.Qt.Horizontal)
        slider.setMinimum(0)
        slider.setMaximum(num_divs)
        slider.setValue(init_val)
        if max_val is None and scalars is not None:
            max_val = np.max(scalars)
        if min_val is None and scalars is not None:
            min_val = np.min(scalars)
        if label_txt is not None:
            label = qw.QLabel(f"{label_txt}: {slider_formula(slider.value()):.3f}")
            toolbar.addWidget(label)
        else:
            label = None
                  
        def valuechange_method(slider_value):                        
            float_value = slider_formula(slider_value)
            if actor_name in self.renderer.actors:
                vis = self.renderer.actors[actor_name].GetVisibility()
            else:
                vis = 0
            update_method(float_value)
            self.renderer.actors[actor_name].SetVisibility(vis)
            if label is not None:
                label.setText(f"{label_txt}: {float_value:.3f}")
        slider.valueChanged.connect(valuechange_method)            
        toolbar.addWidget(slider)
        return label, external_update, slider


    # Add a drop down menu
    def generic_menu_toggle(self, actor_name):        
        def return_function():
            actor=self.renderer.actors[actor_name]    
            actor.SetVisibility(1-actor.GetVisibility())
            if actor_name in self.QSliders.keys():
                slider = self.QSliders[actor_name]
                slider.setValue(slider.value()+1)
                slider.setValue(slider.value()-1)
        return return_function

    def rescale_lights_intensity(self, factor):
        for light in self.renderer.lights:
            light.SetIntensity(factor*light.GetIntensity())

In [382]:
pv.close_all()
qp = Qmin_plot(dat)

In [363]:
qp.renderer.actors["director"].SetVisibility(False)
print(qp.renderer.actors["director"].GetVisibility(), qp.visibilities["director"])

0 1


In [366]:
qp.renderer.actors["director"].SetVisibility(1-qp.renderer.actors["director"].GetVisibility())

In [180]:
dir(qtpy.QtGui)

['PYQT4',
 'PYQT5',
 'PYSIDE',
 'PYSIDE2',
 'PythonQtError',
 'QAbstractOpenGLFunctions',
 'QAbstractTextDocumentLayout',
 'QActionEvent',
 'QBackingStore',
 'QBitmap',
 'QBrush',
 'QClipboard',
 'QCloseEvent',
 'QColor',
 'QColorConstants',
 'QColorSpace',
 'QColorTransform',
 'QConicalGradient',
 'QContextMenuEvent',
 'QCursor',
 'QDesktopServices',
 'QDoubleValidator',
 'QDrag',
 'QDragEnterEvent',
 'QDragLeaveEvent',
 'QDragMoveEvent',
 'QDropEvent',
 'QEnterEvent',
 'QExposeEvent',
 'QFileOpenEvent',
 'QFocusEvent',
 'QFont',
 'QFontDatabase',
 'QFontInfo',
 'QFontMetrics',
 'QFontMetricsF',
 'QGlyphRun',
 'QGradient',
 'QGuiApplication',
 'QHelpEvent',
 'QHideEvent',
 'QHoverEvent',
 'QIcon',
 'QIconDragEvent',
 'QIconEngine',
 'QImage',
 'QImageIOHandler',
 'QImageReader',
 'QImageWriter',
 'QInputEvent',
 'QInputMethod',
 'QInputMethodEvent',
 'QInputMethodQueryEvent',
 'QIntValidator',
 'QKeyEvent',
 'QKeySequence',
 'QLinearGradient',
 'QMatrix2x2',
 'QMatrix2x3',
 'QMatrix2x

In [151]:
qp.director_slice_func([1,0,0],[0,0,0])

In [155]:
qp.renderer.actors["director"].SetVisibility(False)